In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
! cp '/content/drive/MyDrive/cs473/lazydata-20221104T020024Z-001.zip' '/content'
from zipfile import ZipFile
zip = ZipFile('/content/lazydata-20221104T020024Z-001.zip')
zip.extractall()

Mounted at /content/drive/


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import cv2
import pickle as pkl
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! pip install wandb
! pip install ray
! pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 29.7 MB/s 
     |████████████████████████████████| 168 kB 69.5 MB/s 
     |████████████████████████████████| 182 kB 79.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 80.1 MB/s 
     |████████████████████████████████| 166 kB 73.8 MB/s 
     |████████████████████████████████| 166 kB 79.7 MB/s 
     |████████████████████████████████| 162 kB 80.3 MB/s 
     |████████████████████████████████| 162 kB 81.9 MB/s 
     |████████████████████████████████| 158 kB 78.9 MB/s 
     |████████████████████████████████| 157 kB 84.1 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 80.4 MB/s 
     |████████████████████████████████| 157 kB 80.5 MB/s 
     |███████████████████████████

In [ ]:
import random
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image, ImageOps
from torch.utils.data import Dataset, DataLoader, IterableDataset
import torchvision.transforms as transforms
import torchvision.transforms as T
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torch.nn.functional import mse_loss
from torchvision import transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_mean_std(gray= False):
  mean_std = {'mean':[], 'std':[]}
  for view in range(3):
    if gray:
      all_out = transforms.ToTensor()(ImageOps.grayscale(Image.open(f'/content/lazydata/train/X/0/rgb/{view}.png')))
      for i in range(1, 3396):
        out = transforms.ToTensor()(ImageOps.grayscale(Image.open(f'/content/lazydata/train/X/{i}/rgb/{view}.png')))
        all_out += out
    else:
      all_out = transforms.ToTensor()(Image.open(f'/content/lazydata/train/X/0/rgb/{view}.png'))
      for i in range(1, 3396):
        out = transforms.ToTensor()(Image.open(f'/content/lazydata/train/X/{i}/rgb/{view}.png'))
        all_out += out
    mean_std['mean'].append(torch.mean(all_out/3396)) 
    mean_std['std'].append(torch.std(all_out/3396))
  return mean_std
mean_std = get_mean_std()


In [ ]:
# Lazy loading
class LazyLoadDataset(Dataset):
    def __init__(self, path, mean_std = mean_std, train = None, transform = None, data_idx = None):
      self.mean_std = mean_std
      self.transform = transform
      path = path + ('train/' if train else 'test/')
      self.pathX = path + 'X/'
      self.pathY = path + 'Y/'
      self.train = train
      if data_idx is None:
          self.data = os.listdir(self.pathX)
      else:
          self.data = data_idx
        
    def __getitem__(self, idx):

        f = self.data[idx]

        # X
        # read rgb images
        path0 = self.pathX + f + '/rgb/0.png'
        path1 = self.pathX + f + '/rgb/1.png'
        path2 = self.pathX + f + '/rgb/2.png'

        img0, img1, img2 = Image.open(path0), Image.open(path1), Image.open(path2)

        # data augmentation
        if self.transform is not None:
          img0 = self.transform(img0)
          img1 = self.transform(img1)
          img2 = self.transform(img2)


        # normalization of RGB
        img0 = transforms.ToTensor()(img0)
        img1 = transforms.ToTensor()(img1) 
        img2 = transforms.ToTensor()(img2) 

        mean0, mean1, mean2 = self.mean_std['mean'][0], self.mean_std['mean'][1], self.mean_std['mean'][2]
        std0, std1, std2 = self.mean_std['std'][0], self.mean_std['std'][1], self.mean_std['std'][2]

        
        img0 = transforms.Normalize(mean0, std0)(img0)
        img1 = transforms.Normalize(mean1, std1)(img1)
        img2 = transforms.Normalize(mean2, std2)(img2)


        # read depth images
        depth = np.load(self.pathX + f + '/depth.npy')/1000
        depth = np.clip(depth, 0.0, 2.0)
        depth0, depth1, depth2 = depth[0, :, :], depth[1, :, :], depth[2, :, :]
        # normalization of depth


        """depth0 = (depth0 - depth0.mean())/ depth0.std()
        depth1 = (depth1 - depth1.mean())/ depth1.std()
        depth2 = (depth2 - depth2.mean())/ depth2.std()"""
        

        # concat info from three views
        input1 = np.concatenate((img0, depth0[None, :, :]),axis = 0)[None, :, :, :]
        input2 = np.concatenate((img1, depth1[None, :, :]),axis = 0)[None, :, :, :]
        input3 = np.concatenate((img2, depth2[None, :, :]),axis = 0)[None, :, :, :]
        combined = torch.from_numpy(np.concatenate((input1, input2, input3), axis = 0))
        # combined dimension: batch * 3 * 4 * 224 * 224 

        # Y
        if self.train:
            Y = torch.from_numpy(np.load(self.pathY + f + '.npy') * 1000)
            return combined, Y

        else:
          field_id = pkl.load(open(self.pathX + f + '/field_id.pkl', 'rb'))
          return combined, field_id

    
    def __len__(self):
        return len(self.data)

In [ ]:
def gray(img):
  return ImageOps.grayscale(img)

def train_val_split(split_ratio = 0.1):
    val_idx = random.sample(range(3396), math.floor(split_ratio * 3396))
    train_idx = list(set(range(3396)).difference(set(val_idx)))
    val_idx, train_idx = [str(i) for i in val_idx], [str(i) for i in train_idx]
    return train_idx, val_idx

train_idx, val_idx = train_val_split()
train_dataset = LazyLoadDataset('/content/lazydata/',mean_std = mean_std, train = True, data_idx = train_idx, transform = gray)
val_dataset = LazyLoadDataset('/content/lazydata/', mean_std = mean_std, train = True, data_idx = val_idx, transform = gray)
print(f'train:{len(train_dataset)}, val:{len(val_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=48, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

"""for i in train_dataloader:
    print(i[0].shape)"""

train:3057, val:339


'for i in train_dataloader:\n    print(i[0].shape)'

# get the mean of the training set and set it to the bias of the last linear layer

In [ ]:
all_y = [i[1] for i in train_dataloader]
all_y = torch.concat((torch.concat(all_y[:-1], axis = 0), all_y[-1]), axis = 0)
y_mean = torch.mean(all_y, axis = 0).to(device)
y_mean
# tensor([ 47.0646,  52.7343, 118.5616,  70.6122,  -0.6648,  95.8811,  59.8242,
        #-45.2541,  49.1242,  56.6229,  35.7834, -37.9923])

KeyboardInterrupt: ignored

**Architecture: use Convnet layers to extract features of different views, concatenate extracted features and pass forward to linear layers.**

In [ ]:
class MyModel(pl.LightningModule):  
  def __init__(self, dropout=0.15,dropout2=0.3, lr = 1e-3, gray = False):

    super().__init__()
    self.lr = lr
    if gray:
      self.first_channel = 2
    else:
      self.first_channel = 4

    
    layers = [
          nn.Dropout2d(dropout),
          # For gray image set '2' otherwise set '4'
          nn.Conv2d(self.first_channel, 6, kernel_size=3,padding=1),
          nn.ReLU(),
          #nn.Conv2d(6, 6, kernel_size=3,padding=1),
          #nn.ReLU()
          ]
    self.block1 = nn.Sequential(*layers)
    layers = [
              nn.MaxPool2d(kernel_size = 2, stride = 2),
              nn.Dropout2d(dropout),
              nn.Conv2d(6, 12, kernel_size=3,padding=1),
              nn.ReLU(),
              #nn.Conv2d(12, 12, kernel_size=3,padding=1),
              #nn.ReLU()
              ]
    self.block2 = nn.Sequential(*layers)
    layers = [
              nn.MaxPool2d(kernel_size = 2, stride = 2),
              nn.Dropout2d(dropout),
              nn.Conv2d(12, 24, kernel_size=3,padding=1),
              nn.ReLU(),
              #nn.Conv2d(24, 24, kernel_size=3,padding=1),
              #nn.ReLU(),
              #nn.Conv2d(24, 24, kernel_size=3,padding=1),
              #nn.ReLU()
              ]
    self.block3 = nn.Sequential(*layers)
    layers = [
              nn.MaxPool2d(kernel_size = 2, stride = 2),
              nn.Dropout2d(dropout),
              nn.Conv2d(24, 48, kernel_size=3,padding=1),
              nn.ReLU(),
              #nn.Conv2d(48, 48, kernel_size=3,padding=1),
              #nn.ReLU(),
              #nn.Conv2d(48, 48, kernel_size=3,padding=1),
              #nn.ReLU()
              ]
    self.block4 = nn.Sequential(*layers)
    layers = [
              nn.MaxPool2d(kernel_size = 2, stride = 2),
              nn.Dropout2d(dropout),
              nn.Conv2d(48, 48,kernel_size=3,padding=1),
              nn.ReLU(),
              #nn.Conv2d(96, 96, kernel_size=3,padding=1),
              #nn.ReLU(),
              #nn.Conv2d(96, 96, kernel_size=3,padding=1),
              #nn.ReLU()
              ]
    self.block5 = nn.Sequential(*layers)
    layers = [
              nn.MaxPool2d(kernel_size = 2, stride = 2),
              nn.Dropout2d(dropout),
              nn.Conv2d(48, 32, kernel_size=3,padding=1),
              nn.ReLU()
              ]
    self.block6 = nn.Sequential(*layers)
    
    layers = [
        nn.Flatten(),
        nn.Linear(3*1568 , 512, bias = False), 
        nn.Dropout(dropout2),
        nn.Sigmoid(), 
    ]
    self.fc = nn.Sequential(*layers)
    self.linear = nn.Linear(512, 12, bias = False)

  def forward(self, x):
      concat_in = torch.cat([x[:,0,:,:,:], x[:,1,:,:,:], x[:,2,:,:,:]])
      x = self.block1(concat_in)
      x = self.block2(x)
      x = self.block3(x)
      x = self.block4(x)
      x = self.block5(x)
      concat_out = self.block6(x)
      
      chunk_size = concat_out.shape[0]//3
      fea1, fea2, fea3 = torch.split(concat_out, chunk_size)
      features = torch.cat((fea1.view(-1, 1568), fea2.view(-1, 1568), fea3.view(-1, 1568)), axis = 1)
      
      out = self.fc(features)

      y_mean = torch.tensor([ 47.0646,  52.7343, 118.5616,  70.6122,  -0.6648,  95.8811,  59.8242,
    -45.2541,  49.1242,  56.6229,  35.7834, -37.9923], requires_grad=False).double().to(device)
    
      output = self.linear(out) + y_mean
      return output

  def training_step(self, batch, batch_idx):
      x, y = batch[0], batch[1]
      self = self.float()
      out = self(x.float())
      loss = torch.sqrt(mse_loss(out, y))
      self.log("train_loss", loss)
      return loss
    
  def validation_step(self, batch, batch_idx):
      x, y = batch[0], batch[1]
      self = self.float()
      out = self(x.float())
      loss = torch.sqrt(mse_loss(out, y))
      self.log('val_loss', loss)

      return loss
  
  def validation_epoch_end(self, outputs):
      #print(f'outputs:{outputs}')
      avg_loss = torch.stack([x for x in outputs]).mean()
      self.log('val_loss', avg_loss)
      return avg_loss
  
  def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr = self.lr)
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
      return optimizer
  
  def test_pred(self, batch, batch_idx):
      x = batch[0]
      out = self(x)
      return out
      
    



In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune
#wandb.login()
os.makedirs('/content/output', exist_ok=True)
# 2301426a232e1ea72e6398e9d453f510f7dc48b5

In [ ]:
from pytorch_lightning.loggers import WandbLogger
model = MyModel(gray = True).to(device)
"""
wandb_logger = WandbLogger(name="Run_chk_gradients_metrics_topo", 
                            project="MyCNN", 
                            log_model="all")
wandb_logger.watch(model, log="all",log_freq=32)
"""

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='/content/output',       # Don not specify dirpath if using WandB to k
    #dirpath = "checkpoints",
    filename='models-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min')


metrics = {"loss": "val_loss"}
tune_callbacks = TuneReportCallback(metrics, on="validation_end")

trainer = pl.Trainer(
    #logger = wandb_logger,
    callbacks = [checkpoint_callback],
    check_val_every_n_epoch = 1,
    max_epochs = 30,
    log_every_n_steps=2,  
    accelerator='auto',
    devices= 'auto')


history = trainer.fit(model=model, train_dataloaders = train_dataloader, val_dataloaders = val_dataloader)
#trainer.validate(model, val_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name   | Type       | Params
--------------------------------------
0 | block1 | Sequential | 114   
1 | block2 | Sequential | 660   
2 | block3 | Sequential | 2.6 K 
3 | block4 | Sequential | 10.4 K
4 | block5 | Sequential | 20.8 K
5 | block6 | Sequential | 13.9 K
6 | fc     | Sequential | 2.4 M 
7 | linear | Linear     | 6.1 K 
--------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.852     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def inference(best_model_path):
  pretrained_model = MyModel.load_from_checkpoint(best_model_path)
  pretrained_model.eval()
  pretrained_model.freeze()

  test_dataset = LazyLoadDataset('/content/lazydata/',mean_std = mean_std, train = False)
  test_dataloader = DataLoader(test_dataset, batch_size= 128, shuffle = False)

  all_output = None
  for i, batch in enumerate(test_dataloader):
    if i == 0:
      all_output = pretrained_model(batch[0])/1000
      all_ids = batch[1]
    else:
      output = pretrained_model(batch[0]) /1000
      all_output = tf.concat([all_output, output], axis=0)
      all_ids = tf.concat([all_ids, batch[1]], axis = 0)
  return all_output, all_ids

best_model_path = "/content/models-epoch=22-val_loss=6.65.ckpt"
out_path = '/content/gy654_submission.csv'
predictions, ids = inference(best_model_path)
titles = [f'FINGER_POS_{i}' for i in range(1, 13)]
submit_df = pd.DataFrame(predictions, columns = titles).astype("float")
submit_df['ID'] = np.array(ids).astype(int)
submit_df = submit_df[['ID'] + titles]
submit_df.to_csv(out_path, index = False)

In [ ]:
submit_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,61.671412,52.512692,110.611751,72.796018,-3.206575,105.350710,90.396052,-51.165080,23.679626,46.412391,78.777631,-12.095608
1,21.371766,54.761779,131.194648,20.142153,-0.306329,133.944674,6.543896,-55.292015,133.038571,42.302169,41.558133,-32.503243
2,28.421610,54.713500,129.639083,48.816879,-2.197484,124.814124,66.829186,-36.141024,14.301703,53.953888,30.473933,-58.556202
3,30.786108,54.323562,128.088599,46.486677,-1.961482,123.606866,43.538743,-53.405327,121.990760,42.572575,67.173484,-20.035173
4,55.576035,53.229701,119.267034,48.016056,-0.482401,125.649800,34.384430,-54.228779,128.162621,54.627949,8.998514,-58.702932
...,...,...,...,...,...,...,...,...,...,...,...,...
674,16.543757,55.038639,131.847670,16.532731,0.141176,134.478787,70.596571,-52.710406,94.835694,53.829591,41.973600,-4.866844
675,24.075498,54.642023,130.418139,41.222145,-1.786577,124.922990,45.095523,-51.302604,119.708045,53.829130,23.043362,-69.919876
676,72.014416,47.240114,84.842636,66.237090,-2.914792,83.742721,63.168626,-48.388350,69.052744,45.892370,3.259741,-33.870729
677,42.190404,53.633281,123.284880,89.702329,-3.939601,66.076127,53.834541,-35.581917,7.841644,78.493512,47.182146,-14.683769


# Hyperparameter search

In [ ]:
def train_robothand(config, num_epochs=10):
    model = MyModel()
    wandb_logger = WandbLogger(name="Run_chk_gradients_metrics_topo",         # WandB Run name
                               project="MyCNN",           # WandB project name
                               log_model=True             # send modelcheckpoint to WandB at end of training 
                              )
    wandb_logger.watch(model, log="all",log_freq=32)


    early_stopping = pl.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", 
            patience=2,
            mode="min")

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor='val_loss',
        every_n_train_steps = 10,
        #dirpath='./mypl',       # Don not specify dirpath if using WandB to k
        dirpath = "checkpoints",
        filename='models-{epoch:02d}-{val_loss:.2f}',
        save_top_k=1,
        mode='min')


    metrics = {"loss": "val_loss"}
    tune_callbacks = TuneReportCallback(metrics, on="validation_end")

    trainer = pl.Trainer(
        logger=wandb_logger,
        callbacks=[checkpoint_callback],
        max_epochs = num_epochs,
        accelerator='auto',
        devices='auto')

    trainer.fit(model=model, train_dataloaders = train_dataloader)
    trainer.validate(model, val_dataloader)

#train_robothand(config, num_epochs=10)

config = {
    "lr": tune.loguniform(1e-4, 1e-3, 1e-2),
    "batch_size": tune.choice([32, 64]),
}


analysis = tune.run(
 tune.with_parameters(train_robothand, num_epochs=10),
 config=config,
 num_samples=3)

# snippet to save lazy data

In [ ]:
import os
import numpy as np
import pickle as pkl
import torch
from torchvision.utils import save_image

trainx = torch.load('/kaggle/input/ml-final/train/train/trainX.pt')
trainy = torch.load('/kaggle/input/ml-final/train/train/trainY.pt')
testx = torch.load('/kaggle/input/ml-final/test/test/testX.pt')

num_train = trainx[0].shape[0]
num_test = testx[0].shape[0]

os.makedirs('./lazydata', exist_ok=True)

# Save train data
os.makedirs('./lazydata/train', exist_ok=True)
os.makedirs('./lazydata/train/X', exist_ok=True)
os.makedirs('./lazydata/train/Y', exist_ok=True)
for i in range(num_train):
    os.makedirs('./lazydata/train/X/{}'.format(i), exist_ok=True)
    # rgb
    os.makedirs('./lazydata/train/X/{}/rgb'.format(i), exist_ok=True)
    for j in range(3):
        save_image(trainx[0][i][j]/255, './lazydata/train/X/{}/rgb/{}.png'.format(i, j))
    # depth
    depth = trainx[1][i].numpy()
    np.save('./lazydata/train/X/{}/depth.npy'.format(i), depth)
    # field id
    pkl.dump(trainx[2][i], open('./lazydata/train/X/{}/field_id.pkl'.format(i), 'wb'))

    y = trainy[0][i].numpy()
    np.save('./lazydata/train/Y/{}.npy'.format(i), y)
print("Saved train data")

# Save test data
os.makedirs('./lazydata/test', exist_ok=True)
os.makedirs('./lazydata/test/X', exist_ok=True)
for i in range(num_test):
    os.makedirs('./lazydata/test/X/{}'.format(i), exist_ok=True)
    # rgb
    os.makedirs('./lazydata/test/X/{}/rgb'.format(i), exist_ok=True)
    for j in range(3):
        save_image(testx[0][i][j]/255, './lazydata/test/X/{}/rgb/{}.png'.format(i, j))
    # depth
    depth = testx[1][i].numpy()
    np.save('./lazydata/test/X/{}/depth.npy'.format(i), depth)
    # field id
    pkl.dump(testx[2][i], open('./lazydata/test/X/{}/field_id.pkl'.format(i), 'wb'))

print("Saved test data")